In [1]:
import numpy as np

In [92]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

class BM(object):
    
    def __init__(self, num_vis_unit, num_hiden_unit):
        self.num_vis_unit = num_vis_unit
        self.num_hiden_unit = num_hiden_unit
        self.W = np.random.random((num_hiden_unit, num_vis_unit))-0.5
        self.c = np.random.random(num_vis_unit)-0.5
        self.b = np.random.random(num_hiden_unit)-0.5

    def forward(self, vis_unit_val = None):
        if not isinstance(vis_unit_val, np.ndarray):
            vis_unit_val = (np.random.random(self.num_vis_unit) > 0.5).astype(np.int64)
        random_sample = np.random.random(self.num_hiden_unit)
        hiden_unit_val = (sigmoid(self.W.dot(vis_unit_val + self.c) + self.b) > random_sample).astype(np.int64)
        return hiden_unit_val
    
    def backward(self, hiden_unit_val = None):
        if not isinstance(hiden_unit_val, np.ndarray):
            hiden_unit_val = (np.random.random(self.num_hiden_unit) > 0.5).astype(np.int64)
        random_sample = np.random.random(self.num_vis_unit)
        vis_unit_val = (sigmoid(self.W.T.dot(hiden_unit_val + self.b) + self.c) > random_sample).astype(np.int64)
        return vis_unit_val
    
    def learning_step(self, vis_unit_val, lr=0.01):
        hiden_unit_val = self.forward(vis_unit_val)
        gibbs_sample_of_visual_unit_val = self.backward(hiden_unit_val)
        gibbs_sample_of_hidden_unit_val = self.forward(gibbs_sample_of_visual_unit_val)
        self.W += lr*(vis_unit_val.reshape([-1,1]).dot(vis_unit_val.reshape([1,-1])) - 
                     gibbs_sample_of_hidden_unit_val.reshape([-1,1]).dot(gibbs_sample_of_visual_unit_val.reshape([1,-1])))
        self.c += lr*(vis_unit_val - gibbs_sample_of_visual_unit_val)
        self.b += lr*(hiden_unit_val - gibbs_sample_of_hidden_unit_val)

In [93]:
import time
bm = BM(3,2)

In [94]:
train_sample = np.array([1,0,1])
vis_unit_val = []
hiden_unit_val = []
start_learnin = 0 
for i in range(int(1e10)):
    hiden_unit_val.append(bm.forward())
    vis_unit_val.append(bm.backward())
    if (i+1)%1000 == 0 and start_learnin == 1:
        bm.learning_step(train_sample, lr=0.1)
    if (i+1)%100000 == 0:
        start_learnin = 1
        print('hidden mean: ', np.mean(hiden_unit_val, axis=0), 
             'visual mean: ', np.mean(vis_unit_val, axis=0))
        vis_unit_val = []
        hiden_unit_val = []

hidden mean:  [0.62349 0.5843 ] visual mean:  [0.51388 0.64483 0.43844]


ValueError: operands could not be broadcast together with shapes (3,3) (2,3) 

In [85]:
bm.W

array([[ 1.55304021, -0.10451253,  0.653806  ],
       [-0.43809631,  0.12168953, -0.08764883],
       [ 1.20854802, -0.28799984,  0.40310511]])

In [86]:
bm.c

array([ 0.33800118, -0.5160964 ,  0.2042567 ])

In [87]:
bm.b

array([ 0.04297527, -0.28843591,  0.28309701])